# Importing Necessary Modules

In [13]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D
sb.set()

# Reading CSV File

In [14]:
fashiondata_raw = pd.read_csv('fashion_data_2018_2022.csv')
fashiondata_raw.head()

,product_id,product_name,gender,category,pattern,color,age_group,season,price,material,sales_count,reviews_count,average_rating,out_of_stock_times,brand,discount,last_stock_date,wish_list_count,month_of_sale,year_of_sale
0,1001,Biker Jacket,Male,Shirt,Geometric,White,25-35,Spring,70.36,Synthetic,75,65,4.9,3,ZARA,20%,2018/1/28,211,1,2018
1,1002,Business Shirt,Male,Jacket,Polka Dots,Beige,18-24,Summer,91.59,Wool,296,25,3.5,6,ZARA,0%,2018/1/28,211,1,2018
2,1003,Wool Jacket,Female,Dress,Geometric,Brown,18-24,All,129.52,Wool,50,32,4.3,4,ZARA,5%,2018/1/7,64,1,2018
3,1004,Summer Dress,Male,Shorts,Plain,White,25-35,All,116.01,Cotton,435,73,4.6,3,ZARA,10%,2018/1/1,171,1,2018
4,1005,Casual Jeans,Male,Shirt,Plain,Beige,35-45,Winter,125.48,Viscose,79,36,3.6,4,ZARA,0%,2018/1/28,126,1,2018


# Extracting Necessary Data

In [15]:
fashiondata_extracted = pd.DataFrame(fashiondata_raw[["gender", "color", "price", "material", "year_of_sale", "age_group", "season", "category", "sales_count", "average_rating"]])
display(fashiondata_extracted)

,gender,color,price,material,year_of_sale,age_group,season,category,sales_count,average_rating
0,Male,White,70.36,Synthetic,2018,25-35,Spring,Shirt,75,4.9
1,Male,Beige,91.59,Wool,2018,18-24,Summer,Jacket,296,3.5
2,Female,Brown,129.52,Wool,2018,18-24,All,Dress,50,4.3
3,Male,White,116.01,Cotton,2018,25-35,All,Shorts,435,4.6
4,Male,Beige,125.48,Viscose,2018,35-45,Winter,Shirt,79,3.6
...,...,...,...,...,...,...,...,...,...,...
655,Female,Green,48.45,Wool,2022,25-35,Summer,Jacket,328,5.0
656,Female,White,61.52,Cotton,2022,18-24,Winter,Dress,200,4.7
657,Female,White,31.07,Synthetic,2022,35-45,Autumn,Shoes,478,4.0
658,Male,Red,139.69,Cotton Blend,2022,18-24,Winter,Shirt,125,4.8


# Statistical Representation of Numeric Predictors

In [16]:
fashiondata_extracted[['price','year_of_sale', 'sales_count', 'average_rating']].describe().round(2)

,price,year_of_sale,sales_count,average_rating
count,660.00,660.00,660.00,660.00
mean,84.97,2020.00,265.55,4.02
std,37.62,1.42,132.33,0.55
min,20.25,2018.00,50.00,3.00
25%,53.20,2019.00,149.00,3.60
50%,82.04,2020.00,257.00,4.00
75%,117.77,2021.00,378.50,4.50
max,149.93,2022.00,500.00,5.00


# Overview of Categorical Predictors

In [17]:
fashiondata_extracted[['gender', 'age_group', 'color','material', 'season', 'category']].describe()

,gender,age_group,color,material,season,category
count,660,660,660,660,660,660
unique,2,3,12,7,5,8
top,Female,18-24,White,Leather,All,Skirt
freq,331,226,174,104,149,99


# Grouping Data into a Pivot Table

In [18]:
# Groups the data and sums the sales count for each combination
df = fashiondata_extracted.groupby(['year_of_sale', 'season', 'category']).sum()

# Forms a pivot table with grouped data
pivoted = pd.pivot_table(df, values='sales_count', index=['year_of_sale', 'season'], columns=['category'])

# Convert the pivot table to integer type
pivoted = pivoted.fillna(0).astype(int)

display(pivoted)

category             Blouse  Dress  Jacket  Jeans  Shirt  Shoes  Shorts  Skirt
year_of_sale season                                                           
2018         All       1337    445     670   1094    691   1393     548   1199
             Autumn       0    558     715    529   1573   1033     234    699
             Spring    2168    466     345      0    743    452    1121   1841
             Summer       0    705    1544   1328      0    994    1521    977
             Winter     979    940    1233   1695    830    668    1424    683
2019         All          0   1551     892    100    177   1908     959   1723
             Autumn     666    152     625   1408    717   1110    1786   1671
             Spring     229   1187     656   1178    610   1390     921    205
             Summer       0    893      65      0    857    320    1325    666
             Winter    2192   1183     673   1109    122   1653     550   1366
2020         All       1240   1137     202    887   1478    961     827   1773
             Autumn     965    566     649    913   1423    429     851    707
             Spring    1008   1091     549   1108   1717    838    1418    352
             Summer     234    463     835    251    890   1082    1411    726
             Winter    1012    445     570   1706    875    346     190   1525
2021         All       2146    972    2580      0   1370   1106     935    888
             Autumn     752    258     617   1050    699    882     777   1488
             Spring     709   1175       0    281    279    532     535   1511
             Summer     614    952     934    874    726   1462     174   1447
             Winter     434   1074    1254    318    681    699     703    677
2022         All        288   1523     239   1401   1472    762     704   1140
             Autumn    1355   1203     361    434    249   1786     850   1703
             Spring    1576    660    1628   2659    567   1133     630      0
             Summer      59    146     775   1300   1302    353     583   1332
             Winter      69   1301     618    399    614      0    1534    172

# Summary Statistics for Sales Count

In [19]:
pivoted.describe().round(2)

category,Blouse,Dress,Jacket,Jeans,Shirt,Shoes,Shorts,Skirt
count,25.00,25.00,25.00,25.00,25.00,25.00,25.00,25.00
mean,801.28,841.84,769.16,880.88,826.48,931.68,900.44,1058.84
std,704.43,408.67,553.58,653.93,481.09,490.50,444.48,550.83
min,0.00,146.00,0.00,0.00,0.00,0.00,174.00,0.00
25%,229.00,466.00,549.00,318.00,610.00,532.00,583.00,683.00
50%,709.00,940.00,656.00,913.00,726.00,961.00,850.00,1140.00
75%,1240.00,1175.00,892.00,1300.00,1302.00,1133.00,1325.00,1511.00
max,2192.00,1551.00,2580.00,2659.00,1717.00,1908.00,1786.00,1841.00


# Pivot Table for Total Sales

In [20]:
# Sums the sales count for each categories in each season for each year
summed_sales = pivoted.sum(axis=1)

# Create a new DataFrame with the summed sales count
finaldataset = pd.DataFrame(summed_sales, columns=['total_sales'])

# Display the table
display(finaldataset)

total_sales
year_of_sale season             
2018         All            7377
             Autumn         5341
             Spring         7136
             Summer         7069
             Winter         8452
2019         All            7310
             Autumn         8135
             Spring         6376
             Summer         4126
             Winter         8848
2020         All            8505
             Autumn         6503
             Spring         8081
             Summer         5892
             Winter         6669
2021         All            9997
             Autumn         6523
             Spring         5022
             Summer         7183
             Winter         5840
2022         All            7529
             Autumn         7941
             Spring         8853
             Summer         5850
             Winter         4707

# Summary Statistics for Total Sales Count

In [21]:
finaldataset.describe().round(2)

,total_sales
count,25.00
mean,7010.60
std,1431.35
min,4126.00
25%,5892.00
50%,7136.00
75%,8081.00
max,9997.00


# Summary Statistics for Total Sales Count for Each Year

In [23]:
for x in range (2018, 2023):
    print(f"\nYear {x}:")
    display(yearly_data[x].describe().round(2))


Year 2018:


,total_sales
count,5.00
mean,7075.00
std,1117.79
min,5341.00
25%,7069.00
50%,7136.00
75%,7377.00
max,8452.00



Year 2019:


,total_sales
count,5.00
mean,6959.00
std,1833.05
min,4126.00
25%,6376.00
50%,7310.00
75%,8135.00
max,8848.00



Year 2020:


,total_sales
count,5.00
mean,7130.00
std,1110.55
min,5892.00
25%,6503.00
50%,6669.00
75%,8081.00
max,8505.00



Year 2021:


,total_sales
count,5.00
mean,6913.00
std,1901.53
min,5022.00
25%,5840.00
50%,6523.00
75%,7183.00
max,9997.00



Year 2022:


,total_sales
count,5.00
mean,6976.00
std,1671.56
min,4707.00
25%,5850.00
50%,7529.00
75%,7941.00
max,8853.00
